# Part 2 of 2 (OPTIONAL): An extreme case of regular expression processing

> This part is **OPTIONAL**. That is, while there are exercises, they are worth 0 points each. Rather, this notebook is designed for those of you who may have a deeper interest in computational aspects of the course material and would like to explore that.

There is a beautiful theory underlying regular expressions, and efficient regular expression processing is regarded as one of the classic problems of computer science. In the last part of this lab, you will explore a bit of that theory, albeit by experiment.

In particular, the code cells below will walk you through a simple example of the potentially **hidden cost** of regular expression parsing. And if you really want to geek out, look at the article on which this example is taken: https://swtch.com/~rsc/regexp/regexp1.html

In [1]:
import sys
print("* Python version:", sys.version)

* Python version: 3.7.5 (default, Dec 18 2019, 06:24:58) 
[GCC 5.5.0 20171010]


## Quick review

**Exercise 0** (ungraded) Let $a^n$ be a shorthand notation for a string in which $a$ is repeated $n$ times. For example, $a^3$ is the same as $aaa$ and $a^6$ is the same as $aaaaaa$. Write a function to generate the string for $a^n$, given a string $a$ and an integer $n \geq 1$.

In [2]:
def rep_str (a, n):
    """Returns a string consisting of an input string repeated a given number of times."""
    assert type(a) is str and n >= 1
    ###
    return a*n
    ###


In [3]:
# Test cell: `rep_str_test`

def check_fixed(a, n, ans):
    msg = "Testing: '{}'^{} -> '{}'".format(a, n, ans)
    print(msg)
    assert rep_str(a, n) == ans, "Case failed!"
    
check_fixed('a', 3, 'aaa')
check_fixed('cat', 4, 'catcatcatcat')
check_fixed('', 100, '')

def check_rand():
    from random import choice, randint
    a = ''.join([choice([chr(k) for k in range(ord('a'), ord('z')+1)]) for _ in range(randint(1, 5))])
    n = randint(1, 10)
    msg = "Testing: '{}'^{}".format(a, n)
    print(msg)
    s_you = rep_str(a, n)
    for k in range(0, n*len(a), len(a)):
        assert s_you[k:(k+len(a))] == a, "Your result, '{}', is not correct at position {} [{}].".format(s_you, k)
    
for _ in range(10):
    check_rand()

print("\n(Passed!)")

Testing: 'a'^3 -> 'aaa'
Testing: 'cat'^4 -> 'catcatcatcat'
Testing: ''^100 -> ''
Testing: 'hi'^5
Testing: 'fhb'^3
Testing: 'h'^1
Testing: 'jb'^3
Testing: 'r'^4
Testing: 'tklv'^10
Testing: 'mste'^10
Testing: 'gs'^4
Testing: 'ohf'^9
Testing: 'vkyh'^9

(Passed!)


## An initial experiment

Intuitively, you should expect (or hope) that the time to determine whether a string of length $n$ matches a given pattern will be proportional to $n$. Let's see if this holds when matching simple input strings of repeated letters against a pattern designed to match such strings.

In [4]:
import re

In [5]:
# Set up an input problem
n = 3
s_n = rep_str ('a', n) # Input string
pattern = '^a{%d}$' % n # Pattern to match it exactly
##string formatingg: %s acts a placeholder for a string while %d acts as a placeholder for a number.
#n 是在d的位子

# Test it
print ("Matching input '{}' against pattern '{}'...".format (s_n, pattern))
assert re.match (pattern, s_n) is not None

# Benchmark it & report time, normalized to 'n'
timing = %timeit -q -o re.match (pattern, s_n)
t_avg = sum (timing.all_runs) / len (timing.all_runs) / timing.loops / n * 1e9
print ("Average time per match per `n`: {:.1f} ns".format (t_avg))

Matching input 'aaa' against pattern '^a{3}$'...
Average time per match per `n`: 266.9 ns


In [7]:
print(s_n)
print(timing.all_runs)
print(sum(timing.all_runs))
print(len(timing.all_runs))
print(timing.loops)

aaa
[0.8000854249999634, 0.803885452999964, 0.79157909099996, 0.7905585330000235, 0.8010264770000504, 0.8180298830000083, 0.8004903949999971]
5.605655256999967
7
1000000


In [10]:
timing

<TimeitResult : 801 ns ± 8.43 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)>

In [9]:
timing1 = %timeit re.match (pattern, s_n)
timing1

821 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Before moving on, be sure you understand what the above benchmark is doing. For more on the Jupyter "magic" command, `%timeit`, see: http://ipython.readthedocs.io/en/stable/interactive/magics.html?highlight=magic#magic-magic

**Exercise 1** (ungraded) Repeat the above experiment for various values of `n`. To help keep track of the results, feel free to create new code cells that repeat the benchmark for different values of `n`. Explain what you observe. Can you conclude that matching simple regular expression patterns of the form `^a{n}$` against input strings of the form $a^n$ does, indeed, scale linearly?

In [12]:
# Use this code cell (and others, if you wish) to set up an experiment
# to test whether matching simple patterns behaves at worst linearly
# in the length of the input.

###
ns = [2,10,100,1000,10000]
s_n_results = []
for i in ns:
    s_n_results.append(rep_str('a',i))
s_n_results
###


['aa',
 'aaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [13]:
pattern_results = []
for i in ns:
    pattern_results.append('^a{%d}$' %i)
pattern_results

['^a{2}$', '^a{10}$', '^a{100}$', '^a{1000}$', '^a{10000}$']

In [18]:
# Test it
for i in range(len(s_n_results)):
    #print ("Matching input '{}' against pattern '{}'...".format (s_n_results[i], pattern_results[i]))
    assert re.match (pattern_results[i], s_n_results[i]) is not None
    
    # Benchmark it & report time, normalized to 'n'
    timing = %timeit -q -o re.match (pattern_results[i], s_n_results[i])
    t_avg = sum (timing.all_runs) / len (timing.all_runs) / timing.loops / n * 1e9
    print ("Average time per match per `n`: {:.1f} ns".format (t_avg))

Average time per match per `n`: 284.3 ns
Average time per match per `n`: 285.8 ns
Average time per match per `n`: 331.5 ns
Average time per match per `n`: 718.3 ns
Average time per match per `n`: 4669.3 ns


**Answer.** To see asymptotically linear behavior, you'll need to try some fairly large values of $n$, e.g., a thousand, ten thousand, a hundred thousand, and a million. Even then, it may **appear** as though the time continues to decrease, but that does not mean you have not reached an asymptote; why not?

> Regarding the latter question, suppose matching time as a function of input size is $t(n) = \alpha + \beta n$, so that the time per match per $n$ is $t(n)/n$.

## A more complex pattern

Consider a regular expression of the form:

$$(a?)^n(a^n) \quad$$

For instance, $n=3$, the regular expression pattern is `(a?){3}a{3} == a?a?a?aaa`. Start by convincing yourself that an input string of the form,

$$a^n = \underbrace{aa\cdots a}_{n \mbox{ occurrences}}$$

should match this pattern. Here is some code to set up an experiment to benchmark this case.

In [15]:
def setup_inputs(n):
    """Sets up the 'complex pattern example' above."""
    s_n = rep_str('a', n)
    p_n = "^(a?){%d}(a{%d})$" % (n, n)
    print ("[n={}] Matching pattern '{}' against input '{}'...".format(n, p_n, s_n))
    assert re.match(p_n, s_n) is not None
    return (p_n, s_n)

n = 3
p_n, s_n = setup_inputs(n)
timing = %timeit -q -o re.match(p_n, s_n)
t_n = sum(timing.all_runs) / len(timing.all_runs) / timing.loops / n * 1e9
print ("==> Time per run per `n`: {} ns".format(t_n))

[n=3] Matching pattern '^(a?){3}(a{3})$' against input 'aaa'...
==> Time per run per `n`: 408.9629496666562 ns


**Exercise 3** (ungraded) Repeat the above experiment but for different values of $n$, such as $n \in \{3, 6, 9, 12, 15, 18\}$. As before, feel free to use the code cell below or make new code cells to contain the code for your experiments. Summarize what you observe. How does the execution time vary with $n$? Can you explain this behavior?

In [16]:
# Use this code cell (and others, if you wish) to set up an experiment
# to test whether matching simple patterns behaves at worst linearly
# in the length of the input.

###
ns = [3,6,9,12,15,18]
for i in ns:
    p_n,s_n = setup_inputs(i)
    timing = %timeit -q -o re.match(p_n, s_n)
    t_n = sum(timing.all_runs) / len(timing.all_runs) / timing.loops / n * 1e9
    print ("==> Time per run per `n`: {} ns".format(t_n))
###
#running time grows exponentially

[n=3] Matching pattern '^(a?){3}(a{3})$' against input 'aaa'...
==> Time per run per `n`: 402.7275427142957 ns
[n=6] Matching pattern '^(a?){6}(a{6})$' against input 'aaaaaa'...
==> Time per run per `n`: 1355.2368523807802 ns
[n=9] Matching pattern '^(a?){9}(a{9})$' against input 'aaaaaaaaa'...
==> Time per run per `n`: 8824.963652380444 ns
[n=12] Matching pattern '^(a?){12}(a{12})$' against input 'aaaaaaaaaaaa'...
==> Time per run per `n`: 68197.60285714523 ns
[n=15] Matching pattern '^(a?){15}(a{15})$' against input 'aaaaaaaaaaaaaaa'...
==> Time per run per `n`: 551329.4389524097 ns
[n=18] Matching pattern '^(a?){18}(a{18})$' against input 'aaaaaaaaaaaaaaaaaa'...
==> Time per run per `n`: 4360200.089523849 ns


**Answer.** Here, you should observe something more like polynomial growth. Here are some results we collected, for instance.

|    n    |  t (ns)   |
|---------|-----------|
|       3 |     412.3 |
|       6 |     728.4 |
|       9 |   3,259.1 |
|      12 |  20,201.9 |
|      15 | 131,392.2 |
|      18 | 861,721.7 |

**Fin!** This cell marks the end of Part 2, which is the final part of this assignment. Don't forget to save, restart and rerun all cells, and submit it.